In [7]:
import pandas, datetime
from pyspark.sql import functions as F
#initial date range
begindate = '2010-01-01'
enddate = '2030-12-31'
df_panda = pandas.DataFrame({'DateTime':pandas.date_range(start=begindate, end=enddate)})
df_panda['Date'] = pandas.to_datetime(df_panda['DateTime']).dt.date

df = spark.createDataFrame(df_panda)

StatementMeta(, a5b9ae49-87bc-448e-b013-0525db9aacc4, 9, Finished, Available)

In [8]:
# Get the current date and time to calculate relative columns
current_datetime = datetime.datetime.now()

# Extract the date component and add to df
current_date = current_datetime.date()
df = df.withColumn ("CurrentDate", F.lit(current_date))

StatementMeta(, a5b9ae49-87bc-448e-b013-0525db9aacc4, 10, Finished, Available)

In [9]:
# add derived columns, weeknumber follows ISO-8601 (week 1 is first week with at least 4 days)
df.createOrReplaceTempView("DATES")
df_sql = spark.sql("""
SELECT 
year(Date) * 10000 + month(Date) * 100 + day(Date) as dateInt, Date, 
year(date) as year, year(date) - year(CurrentDate) as RelativeYear,
quarter(date) as quarter, concat("Q", quarter(date)) as QuarterName, quarter(date) -quarter(CurrentDate) as RelativeQuarter,
concat(year(date), "-", "Q", quarter(date)) as YearQuarter,
month(date) as month, month(date) - month(CurrentDate) as RelativeMonth, date_format(Date, 'MMMM') as MonthName, date_format(Date, 'MMM') as MonthNameShort,
year(Date) * 100 + month(Date) as MonthYear, concat(date_format(Date, 'MMM'), "'", date_format(Date, 'yy')) as MonthYearName,
weekofyear(date) as WeekOfYear, concat("week ", weekofyear(date)) as WeekOfYearName,
case when weekofyear(date) = 53 and month(Date) = 1 then year(date) -1 else year(date) end as ISOYear,
case when weekofyear(date) = 53 and month(Date) = 1 then year(date) -1 else year(date) end * 100 + weekofyear(Date) as ISOYearWeek,
year(date) * 100 +  weekofyear(Date) as YearWeek,
day(date) as day, dayofweek(Date) AS DayOfWeekStartSunday, weekday(Date) + 1 as DayOfWeekStartMonday, 
date_format(Date, 'EEE') as WeekdayNameShort, date_format(Date, 'EEEE') as WeekdayName,
case when weekday(Date) < 5 then "Y" else "N" end as IsWeekDay,
case when weekday(Date) < 5 then "N" else "Y" end as IsWeekend,
case when year(date) = year(CurrentDate) then 'Current' else year(date) end as DefaultYear,
case when month(date) = month(CurrentDate) then 'Current' else date_format(Date, 'MMM') end as DefaultMonth,
case when quarter(date) = quarter(CurrentDate) then 'Current' else concat("Q", quarter(date)) end as DefaultQuarter
FROM DATES
""")
#display(df_sql)


StatementMeta(, a5b9ae49-87bc-448e-b013-0525db9aacc4, 11, Finished, Available)

In [10]:
# Delete old Date Table and Create New
delta_table_name = "CalendarDates"
df_sql.write.format("delta").mode("overwrite").saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")

StatementMeta(, a5b9ae49-87bc-448e-b013-0525db9aacc4, 12, Finished, Available)

DataFrame has been written to Delta table: CalendarDates
